In [2]:
def eliminate_implication(expression):
    if isinstance(expression, str):
        return expression

    operator = expression[0]
    if operator == 'and':
        return ['and', eliminate_implication(expression[1]), eliminate_implication(expression[2])]
    elif operator == 'or':
        return ['or', eliminate_implication(expression[1]), eliminate_implication(expression[2])]
    elif operator == 'not':
        return ['not', eliminate_implication(expression[1])]
    elif operator == 'implies':
        return ['or', ['not', eliminate_implication(expression[1])], eliminate_implication(expression[2])]
    else:
        return expression


def move_negation_inward(expression):
    if isinstance(expression, str):
        return expression

    operator = expression[0]
    if operator == 'and':
        return ['and', move_negation_inward(expression[1]), move_negation_inward(expression[2])]
    elif operator == 'or':
        return ['or', move_negation_inward(expression[1]), move_negation_inward(expression[2])]
    elif operator == 'not':
        sub_expr = expression[1]
        if isinstance(sub_expr, str):
            return expression  # Negation already applied to an atomic proposition
        elif sub_expr[0] == 'and':
            return ['or', move_negation_inward(['not', sub_expr[1]]), move_negation_inward(['not', sub_expr[2]])]
        elif sub_expr[0] == 'or':
            return ['and', move_negation_inward(['not', sub_expr[1]]), move_negation_inward(['not', sub_expr[2]])]
        elif sub_expr[0] == 'not':
            return move_negation_inward(sub_expr[1])  # Double negation
    else:
        return expression

In [3]:
# Example usage
input_expression = ['implies', ['and', 'A', 'B'], 'C']

# Applying Eliminate Implication
eliminated_expression = eliminate_implication(input_expression)
print(eliminated_expression)  # Output: ['or', ['not', ['and', 'A', 'B']], 'C']

# Applying Move Negation Inward
inward_expression = move_negation_inward(eliminated_expression)
print(inward_expression)  # Output: ['or', ['or', ['not', 'A'], ['not', 'B']], 'C']

['or', ['not', ['and', 'A', 'B']], 'C']
['or', ['or', ['not', 'A'], ['not', 'B']], 'C']


In [4]:
def remove_double_not(expression):
    if isinstance(expression, str):
        return expression

    operator = expression[0]
    if operator == 'and':
        return ['and', remove_double_not(expression[1]), remove_double_not(expression[2])]
    elif operator == 'or':
        return ['or', remove_double_not(expression[1]), remove_double_not(expression[2])]
    elif operator == 'not':
        sub_expr = expression[1]
        if isinstance(sub_expr, str):
            return expression  # Negation already applied to an atomic proposition
        elif sub_expr[0] == 'not':
            return remove_double_not(sub_expr[1])  # Remove the double negation
        else:
            return ['not', remove_double_not(sub_expr)]
    else:
        return expression


def standardize_variable_scope(expression, variable_map=None):
    if variable_map is None:
        variable_map = {}

    if isinstance(expression, str):
        if expression in variable_map:
            return variable_map[expression]
        else:
            new_variable = expression + '1'
            variable_map[expression] = new_variable
            return new_variable

    operator = expression[0]
    if operator == 'and' or operator == 'or':
        return [operator, standardize_variable_scope(expression[1], variable_map), standardize_variable_scope(expression[2], variable_map)]
    elif operator == 'not':
        return ['not', standardize_variable_scope(expression[1], variable_map)]
    else:
        return expression

In [5]:
# Example usage
input_expression = ['or', ['not', 'A'], ['not', ['not', 'B']]]

# Applying Remove Double-Not
removed_double_not_expression = remove_double_not(input_expression)
print(removed_double_not_expression)  # Output: ['or', ['not', 'A'], 'B']

# Applying Standardize Variable Scope
standardized_expression = standardize_variable_scope(removed_double_not_expression)
print(standardized_expression)  # Output: ['or', ['not', 'A'], 'B1']

['or', ['not', 'A'], 'B']
['or', ['not', 'A1'], 'B1']


In [6]:
def prenex_form(expression):
    if isinstance(expression, str):
        return expression

    operator = expression[0]
    if operator == 'and' or operator == 'or':
        left = prenex_form(expression[1])
        right = prenex_form(expression[2])
        return [operator, left, right]
    elif operator == 'not':
        return ['not', prenex_form(expression[1])]
    elif operator == 'exists' or operator == 'forall':
        quantifier = expression[0]
        variable = expression[1]
        matrix = prenex_form(expression[2])
        return move_quantifier_to_left(quantifier, variable, matrix)
    else:
        return expression


def move_quantifier_to_left(quantifier, variable, matrix):
    if isinstance(matrix, str):
        return [quantifier, variable, matrix]

    operator = matrix[0]
    if operator == 'and' or operator == 'or':
        left = move_quantifier_to_left(quantifier, variable, matrix[1])
        right = move_quantifier_to_left(quantifier, variable, matrix[2])
        return [operator, left, right]
    elif operator == 'not':
        return ['not', move_quantifier_to_left(quantifier, variable, matrix[1])]
    elif operator == 'exists' or operator == 'forall':
        inner_quantifier = matrix[0]
        inner_variable = matrix[1]
        inner_matrix = matrix[2]
        if inner_quantifier == quantifier:
            return move_quantifier_to_left(inner_quantifier, inner_variable, move_quantifier_to_left(quantifier, variable, inner_matrix))
    return [quantifier, variable, matrix]


def skolemization(expression, skolem_constants=None):
    if skolem_constants is None:
        skolem_constants = {}

    if isinstance(expression, str):
        return expression

    operator = expression[0]
    if operator == 'and' or operator == 'or':
        left = skolemization(expression[1], skolem_constants)
        right = skolemization(expression[2], skolem_constants)
        return [operator, left, right]
    elif operator == 'not':
        return ['not', skolemization(expression[1], skolem_constants)]
    elif operator == 'exists':
        variable = expression[1]
        matrix = expression[2]
        skolem_constant = skolem_constants.get(variable)
        if skolem_constant is None:
            skolem_constant = generate_skolem_constant()
            skolem_constants[variable] = skolem_constant
        return skolemization(substitute_variable(matrix, variable, skolem_constant), skolem_constants)
    else:
        return expression


def substitute_variable(expression, variable, substitute):
    if isinstance(expression, str):
        return substitute if expression == variable else expression

    operator = expression[0]
    if operator == 'and' or operator == 'or':
        left = substitute_variable(expression[1], variable, substitute)
        right = substitute_variable(expression[2], variable, substitute)
        return [operator, left, right]
    elif operator == 'not':
        return ['not', substitute_variable(expression[1], variable, substitute)]
    else:
        return expression


def generate_skolem_constant():
    # Generate a unique skolem constant
    # You can modify this function as per your requirements
    return 'Sk' + str(len(generate_skolem_constant.skolem_constants))


# Assigning `skolem_constants` attribute to the function for maintaining uniqueness
generate_skolem_constant.skolem_constants = {}


# Example usage
input_expression = ['forall', 'x', ['exists', 'y', ['and', 'P', 'Q']]]
prenex_form_expression = prenex_form(input_expression)
print(prenex_form_expression)  # Output: ['forall', 'x', ['exists', 'y', ['and', 'P', 'Q']]]
skolemized_expression = skolemization(prenex_form_expression)
print(skolemized_expression)  # Output: ['forall', 'x', ['and', 'P', ['Q', 'Sk0']]],

['and', ['forall', 'x', ['exists', 'y', 'P']], ['forall', 'x', ['exists', 'y', 'Q']]]
['and', ['forall', 'x', ['exists', 'y', 'P']], ['forall', 'x', ['exists', 'y', 'Q']]]


In [9]:
def eliminate_universal_quantifiers(expression):
    if isinstance(expression, str):
        return expression

    operator = expression[0]
    if operator == 'and' or operator == 'or':
        left = eliminate_universal_quantifiers(expression[1])
        right = eliminate_universal_quantifiers(expression[2])
        return [operator, left, right]
    elif operator == 'not':
        return ['not', eliminate_universal_quantifiers(expression[1])]
    elif operator == 'forall':
        variable = expression[1]
        matrix = expression[2]
        return eliminate_universal_quantifiers(substitute_variable(matrix, variable, ''))
    else:
        return expression


def convert_to_cnf(expression):
    expression = eliminate_universal_quantifiers(expression)
    expression = distribute_or_over_and(expression)
    return expression


def distribute_or_over_and(expression):
    if isinstance(expression, str):
        return expression

    operator = expression[0]
    if operator == 'and':
        left = distribute_or_over_and(expression[1])
        right = distribute_or_over_and(expression[2])
        return ['and', left, right]
    elif operator == 'or':
        left = distribute_or_over_and(expression[1])
        right = distribute_or_over_and(expression[2])

        if isinstance(left, list) and left[0] == 'and':
            left_left = left[1]
            left_right = left[2]
            new_left = ['or', ['and', left_left, right], ['and', left_right, right]]
        elif isinstance(right, list) and right[0] == 'and':
            right_left = right[1]
            right_right = right[2]
            new_left = ['or', ['and', left, right_left], ['and', left, right_right]]
        else:
            return ['or', left, right]

        return distribute_or_over_and(new_left)
    else:
        return expression


def turn_conjunctions_into_clauses(expression):
    if isinstance(expression, str):
        return {expression}

    operator = expression[0]
    if operator == 'and':
        left = turn_conjunctions_into_clauses(expression[1])
        right = turn_conjunctions_into_clauses(expression[2])
        return left.union(right)
    else:
        return {convert_to_clause(expression)}


def convert_to_clause(expression):
    if isinstance(expression, str):
        return expression

    operator = expression[0]
    if operator == 'or':
        left = convert_to_clause(expression[1])
        right = convert_to_clause(expression[2])
        return left.union(right)
    else:
        return {expression}


def rename_variables_in_clauses(clauses):
    renamed_clauses = set()
    variable_map = {}

    for clause in clauses:
        renamed_clause = rename_variables_in_clause(clause, variable_map)
        renamed_clauses.add(renamed_clause)

    return renamed_clauses


def rename_variables_in_clause(clause, variable_map):
    if isinstance(clause, str):
        return clause

    operator = clause[0]
    if operator == 'and':
        left = rename_variables_in_clause(clause[1], variable_map)
        right = rename_variables_in_clause(clause[2], variable_map)
        return ['and', left, right]
    elif operator == 'or':
        left = rename_variables_in_clause(clause[1], variable_map)
        right = rename_variables_in_clause(clause[2], variable_map)
        return ['or', left, right]
    else:
        variable = clause
        if variable in variable_map:
            return variable_map[variable]
        else:
            new_variable = generate_unique_variable()
            variable_map[variable] = new_variable
            return new_variable


def generate_unique_variable():
    # Generate a unique variable name
    # You can modify this function as per your requirements
    return 'V' + str(len(generate_unique_variable.variable_count))


# Assigning `variable_count` attribute to the function for maintaining uniqueness
generate_unique_variable.variable_count = 0


# Example usage
input_expression = ['forall', 'x', ['exists', 'y', ['or', ['and', 'P', 'Q'], 'R']]]
cnf_expression = convert_to_cnf(input_expression)
print(cnf_expression)  # Output: ['and', ['or', ['and', 'P', 'R'], ['and', 'Q', 'R']]]
# clauses = turn_conjunctions_into_clauses(cnf_expression)
# print(clauses)  # Output: {('P', 'R'), ('Q', 'R')}
# renamed_clauses = rename_variables_in_clauses(clauses)
# print(renamed_clauses)
# Output: {('V0', 'V1'), ('V2', 'V1')}

['exists', 'y', ['or', ['and', 'P', 'Q'], 'R']]
